In [1]:
import os
import numpy as np
import psycopg2
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from psycopg2.extras import DictCursor
from sklearn.metrics.pairwise import cosine_similarity



In [2]:
# save to user_data

connection = psycopg2.connect(host="localhost", user="root", port=5432, database="W9sV6cL2dX", password="E5rG7tY3fH")
cursor = connection.cursor(cursor_factory=DictCursor)

select_query = "SELECT * FROM users WHERE gender IS NOT NULL"
cursor.execute(select_query)
user_data = cursor.fetchall()


In [3]:
data = {}
data['gender'] = [d[1] for d in user_data]
data['country'] = [d[2] for d in user_data]
data['age'] = [d[3] for d in user_data]

data

{'gender': ['male',
  'female',
  'female',
  'male',
  'male',
  'female',
  'male',
  'female',
  'male',
  'male',
  'male',
  'female',
  'male',
  'other',
  'female',
  'male',
  'male',
  'male',
  'female',
  'male',
  'male',
  'female',
  'female',
  'female',
  'female',
  'other',
  'male',
  'female',
  'male',
  'male',
  'male',
  'female',
  'male',
  'female',
  'female',
  'male',
  'female',
  'female',
  'female',
  'female',
  'female',
  'female',
  'male',
  'female',
  'female',
  'female',
  'other',
  'male',
  'female',
  'female',
  'male',
  'male',
  'male',
  'female',
  'female',
  'other',
  'male',
  'female',
  'male',
  'female',
  'male',
  'female',
  'male',
  'male',
  'male',
  'other',
  'other',
  'female',
  'female',
  'female',
  'female',
  'female',
  'female',
  'female',
  'male',
  'male',
  'male',
  'female',
  'female',
  'female',
  'female',
  'male',
  'female',
  'male',
  'male',
  'female',
  'male',
  'female',
  'male',
  'm

In [4]:
select_query = "SELECT DISTINCT country FROM users WHERE country IS NOT NULL;"
select_query_age = "SELECT DISTINCT age FROM users WHERE age IS NOT NULL;"

cursor.execute(select_query)
db_countries = cursor.fetchall()

cursor.execute(select_query_age)
db_age = cursor.fetchall()

# define your categories
genders = ['male', 'female', 'other']
ages = [age[0] for age in db_age]
countries = [country[0] for country in db_countries]

print(genders)
print(ages)
print(countries)

['male', 'female', 'other']
[55, 27, 23, 56, 91, 58, 8, 87, 74, 54, 29, 71, 68, 4, 34, 51, 96, 80, 70, 52, 83, 67, 63, 90, 10, 35, 45, 6, 84, 86, 39, 92, 93, 89, 69, 36, 31, 50, 60, 14, 66, 22, 59, 13, 65, 2, 16, 62, 75, 73, 44, 11, 42, 88, 82, 41, 46, 40, 43, 53, 32, 9, 7, 38, 15, 79, 48, 12, 26, 85, 72, 78, 57, 24, 81, 61, 19, 77, 25, 94, 30, 21, 49, 47, 3, 17, 20, 37, 28, 33, 1, 76, 5, 18, 64]
['FibonacciFlats', 'Neuropolis', 'TensorPeak', 'TuringLake', 'AdaLove', 'AlgoBay', 'BayesianBourg', 'GraphTown']


In [5]:
gender_encoder = LabelEncoder()
gender_encoder.fit(genders)
country_encoder = LabelEncoder()
country_encoder.fit(countries)
# normalise age by subtracting the mean and dividing by the SD
age_scaler = StandardScaler()

# Convert data to tensors
gender_tensor = torch.tensor(gender_encoder.transform(data['gender']), dtype=torch.long)
country_tensor = torch.tensor(country_encoder.transform(data['country']), dtype=torch.long)
age_normalised = torch.tensor(age_scaler.fit_transform(np.array(data['age']).reshape(-1, 1)), dtype=torch.float32)

# # Define the embedding layers for gender and country
gender_embed = nn.Embedding(len(genders), 1)  # 10 is the embedding dimension
country_embed = nn.Embedding(len(countries), 1)  # 10 is the embedding dimension

# Now you can transform the data
gender_encoded = gender_embed(gender_tensor)
country_encoded = country_embed(country_tensor)

print("Gender Encoded: ", gender_tensor.shape)
print("Country Encoded: ", country_tensor.shape)
print("Age Normalized: ", len(age_normalised))

Gender Encoded:  torch.Size([40000])
Country Encoded:  torch.Size([40000])
Age Normalized:  40000


In [14]:
user_embeddings = torch.cat((gender_encoded, country_encoded, age_normalised), dim=1)
user_embeddings.shape # torch.Size([40000, 3]) which is 40K users and 3 features 
select_query = "SELECT embedding FROM item_embeddings;"
cursor.execute(select_query)
item_embeddings_db = cursor.fetchall()
item_embeddings = torch.tensor(item_embeddings_db)


In [20]:
item_embeddings.shape

# Let's assume we're training on a subset of 20,000 user-item pairs as we pass in 2000 items and 1000 users to model
num_pairs = 200

# Generate random expected interactions for these pairs
# We will assume that 0 means the user did not interact with the item (spent less than 5s)
# and 1 means the user did interact with the item (spent more than 5s).
expected_interactions = np.random.randint(2, size=num_pairs)
# Convert to a tensor
expected_interactions = torch.tensor(expected_interactions, dtype=torch.float32).unsqueeze(1)

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TwoTowerNetwork(nn.Module):
    def __init__(self, user_input_dim, item_input_dim, output_dim):
        super(TwoTowerNetwork, self).__init__()

        hidden_dim = 128  # example value, adjust based on your requirement

        # User tower
        self.user_fc1 = nn.Linear(user_input_dim, hidden_dim)
        self.user_fc2 = nn.Linear(hidden_dim, output_dim)

        # Item tower
        self.item_fc1 = nn.Linear(item_input_dim, hidden_dim)
        self.item_fc2 = nn.Linear(hidden_dim, output_dim)

        self.relu = nn.ReLU()

    def forward(self, user_input, item_input):
        # User tower
        user_output = self.user_fc1(user_input)
        user_output = self.relu(user_output)
        user_output = self.user_fc2(user_output)

        # Item tower
        item_output = self.item_fc1(item_input)
        item_output = self.relu(item_output)
        item_output = self.item_fc2(item_output)

        # Normalize the embeddings (this is necessary for cosine similarity)
        user_output = F.normalize(user_output, dim=1)
        item_output = F.normalize(item_output, dim=1)

        # Compute cosine similarity
        # Cosine similarity is the dot product of the normalized vectors
        cosine_similarity = torch.sum(user_output * item_output, dim=1)

        # Convert the cosine similarity to a probability (between 0 and 1)
        prob = torch.sigmoid(cosine_similarity)

        return prob

model = TwoTowerNetwork(3, 1000, 1)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCELoss()

for epoch in range(3):
    model.train()
    # user_input and item_input are your input tensors, expected_interactions is [1,0,0..] where 1 signifies the user spent more than 5s on an item
    output = model(user_embeddings[:100], item_embeddings[:200])
    loss = criterion(output, expected_interactions)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


In [16]:
print(output.shape)


torch.Size([200, 512])
